<a href="https://colab.research.google.com/github/Sabih-Rahman5/2024-ai-challenge-superray/blob/main/DeepSeek_RAG/RAG_DeepSeek_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Naive RAG with DeepSeek and LangChain

This notebook shows an easy RAG (Retrieval Augmented Generation) with DeepSeek model from Hugging Face [`deepseek-ai/DeepSeek-R1`](https://huggingface.co/deepseek-ai/DeepSeek-R1), and LangChain.


**RAG process**

The RAG (Retrieval-Augmented Generation) system combines a retrieval system with an LLM. The system first retrieves relevant documents from a corpus using a vector database, then uses an LLM hosted in Hugging Face to generate answers based on the retrieved documents.


# Prepare Workspace

In [1]:
!pip install -q torch transformers sentence-transformers faiss-cpu pypdf &> /dev/null

In [2]:
!pip install -U langchain-huggingface &>/dev/null

In [3]:
!pip install -q langchain langchain-community &> /dev/null

In [4]:
import langchain as lc
from langchain import LLMMathChain
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_huggingface import HuggingFacePipeline


## Upload the data


In [6]:
# Load content from local PDFs
loader = PyPDFLoader("./2501.12948v1.pdf")
docs = loader.load()

In [7]:
# Define the document:
Document(page_content="DeepSeek-R1:Incentivizing Reasoning Capability in LLMs via Reinforcement Learning.",
         metadata={
             'document_id' : '2501.12948v1',
             'document_source' : "ArXiv",
             'document_create_time' : "2025"
         })

Document(metadata={'document_id': '2501.12948v1', 'document_source': 'ArXiv', 'document_create_time': '2025'}, page_content='DeepSeek-R1:Incentivizing Reasoning Capability in LLMs via Reinforcement Learning.')

In [8]:
print("\nPage Content: ", docs[0].page_content)
print("\nMeta Data: ", docs[0].metadata)


Page Content:  DeepSeek-R1: Incentivizing Reasoning Capability in LLMs via
Reinforcement Learning
DeepSeek-AI
research@deepseek.com
Abstract
We introduce ourrst-generation reasoning models, DeepSeek-R1-Zero and DeepSeek-R1.
DeepSeek-R1-Zero, a model trained via large-scale reinforcement learning (RL) without super-
visedne-tuning (SFT) as a preliminary step, demonstrates remarkable reasoning capabilities.
Through RL, DeepSeek-R1-Zero naturally emerges with numerous powerful and intriguing
reasoning behaviors. However, it encounters challenges such as poor readability, and language
mixing. To address these issues and further enhance reasoning performance, we introduce
DeepSeek-R1, which incorporates multi-stage training and cold-start data before RL. DeepSeek-
R1 achieves performance comparable to OpenAI-o1-1217 on reasoning tasks. To support the
research community, we open-source DeepSeek-R1-Zero, DeepSeek-R1, and six dense models
(1.5B, 7B, 8B, 14B, 32B, 70B) distilled from DeepSee

In [9]:
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=30)
chunked_docs = splitter.split_documents(docs)

In [10]:
print("PDF Splited by Chunks - You have {0} number of chunks.".format(len(docs)))

PDF Splited by Chunks - You have 22 number of chunks.


## Embeddings + Retriever

For embeddings I use the `HuggingFaceEmbeddings` and the [`BAAI/bge-base-en-v1.5`](https://huggingface.co/BAAI/bge-base-en-v1.5) embeddings model.

To create the vector database, I use `FAISS`, a library developed by Facebook AI. This library offers efficient similarity search and clustering of dense vectors.

In [11]:
db = FAISS.from_documents(chunked_docs,
                          HuggingFaceEmbeddings(model_name='BAAI/bge-base-en-v1.5'))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 3}
)

## Load the model

In [13]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

## Setup the RAG

First, I create a text_generation pipeline using the loaded model and its tokenizer.

Next, I create a prompt template.

Then, I combined the `llm_chain` with the retriever to create a RAG chain.

In [14]:
# Pipeline for text generation
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=500,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Prompt template to match desired output format
prompt_template = """
You are a professional AI researcher, give an help in study. Use the following context to answer the question using information provided by the paper:

{context}

Question: {question}
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

llm_chain = prompt | llm | StrOutputParser()


rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)



Device set to use cuda:0


# Questions

In [15]:
question = "What are the advantages of using reinforcement learning directly on a base model, as demonstrated by DeepSeek-R1-Zero?"

# Invoke the chain to generate answers
result = rag_chain.invoke(question)

# Display the output
print(result)

</think>

The advantages of using reinforcement learning directly on a base model, as demonstrated by DeepSeek-R1-Zero, include:

1. **Direct Application of RL**: DeepSeek-R1-Zero showcases that RL can be applied directly to a base model without prior supervised fine-tuning (SFT), enabling it to explore chain-of-thought (CoT) solutions effectively.

2. **Enhanced Problem Solving Capabilities**: By leveraging RL, DeepSeek-R1-Zero demonstrates improved abilities in self-reasoning, reflection, and generation tasks, showcasing its robust problem-solving skills.

3. **Natural Emergence of Reasoning**: The model's effectiveness is attributed to its ability to emerge through RL processes inherently, without the need for extensive pre-training or fine-tuning.

These findings highlight the potential of RL in advancing language models like DeepSeek-R1-Zero, emphasizing their adaptability and efficiency in various computational tasks.


In [16]:
question = "What is cold-start data and why is it used in DeepSeek-R1 training?"

# Invoke the chain to generate answers
result = rag_chain.invoke(question)

# Display the output
print(result)

</think>

Cold-start data refers to the initial training data used when a new model starts learning, often before sufficient experience has been gained through previous interactions or tasks. It plays a crucial role in enabling models like DeepSeek-R1 to perform well initially.

In the case of DeepSeek-R1, cold-start data is used to enhance its ability to learn quickly and effectively. This is achieved by introducing a small amount of high-quality data during the training process, which helps the model adapt more smoothly and efficiently to new challenges.

The use of cold-start data allows DeepSeek-R1 to demonstrate superior performance even when it begins with limited knowledge, making it particularly effective in cold-start scenarios where traditional warm-start approaches may not suffice.


In [17]:
question = "What are DeepSeek-R1-Zero and DeepSeek-R1?"

# Invoke the chain to generate answers
result = rag_chain.invoke(question)

# Display the output
print(result)

</think>

DeepSeek-R1 and DeepSeek-R1-Zero are both AI models developed by DeepSeek. Here's a breakdown of their differences:

1. **DeepSeek-R1**:
   - **Version**: This version was released in March 2025.
   - **Key Features**:
     - Represents a pure RL (Reinforcement Learning) approach without relying on cold-start data.
     - Achieves strong performance across various tasks.
     - Leverages cold-start data alongside iterative RL techniques for improved performance.

2. **DeepSeek-R1-Zero**:
   - **Version**: This is a newer version of DeepSeek-R1, also released in March 2025.
   - **Key Features**:
     - Emphasizes better readability and sharing of reasoning processes with the open community.
     - Utilizes RL with human-friendly cold-start data.

### Summary:
- **DeepSeek-R1** is the original model, focusing on pure RL approaches and leveraging cold-start data.
- **DeepSeek-R1-Zero** is an enhanced version of DeepSeek-R1, improving readability and accessibility while maintain

In [18]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:
<think>{}"""

In [ ]:
question = "A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?"


prompt = [prompt_style.format(question, "")]
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

print(prompt)



In [ ]:
result = rag_chain.invoke(question)



outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])